In [26]:
from IPython.display import Image, HTML, display
from matplotlib import pyplot as plt
import numpy as np 
import os
import cv2
import csv

import math
import argparse
import matplotlib
import imghdr
import pickle as pkl
import datetime
from cycler import cycler
from PIL import Image, ImageEnhance

from pathlib import Path
import os.path

from datasets import load_dataset
from datasets import load_metric

from transformers import TrainingArguments
from transformers import ViTFeatureExtractor
from transformers import ViTForImageClassification

import torch

from PIL import Image
import requests
import numpy as np

In [27]:
!rm -rf sample_data
!wget https://cluster.ig.umons.ac.be/HackIA21/databases/FIRE_DATABASE_3.tar
!tar xf FIRE_DATABASE_3.tar
!rm FIRE_DATABASE_3.tar

--2022-10-30 19:46:04--  https://cluster.ig.umons.ac.be/HackIA21/databases/FIRE_DATABASE_3.tar
Resolving cluster.ig.umons.ac.be (cluster.ig.umons.ac.be)... 193.190.209.220
Connecting to cluster.ig.umons.ac.be (cluster.ig.umons.ac.be)|193.190.209.220|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 471111680 (449M) [application/octet-stream]
Saving to: ‘FIRE_DATABASE_3.tar’

FIRE_DATABASE_3.tar 100%[===================>] 449.29M  22.0MB/s    in 22s     

2022-10-30 19:46:27 (20.2 MB/s) - ‘FIRE_DATABASE_3.tar’ saved [471111680/471111680]



In [28]:
# Données de test
!rm -rf sample_data
!wget --no-check-certificate http://195.154.53.219/downloads/test.tar
! tar xf test.tar -C 'test_data' --one-top-level
! rm test.tar

--2022-10-30 19:46:32--  http://195.154.53.219/downloads/test.tar
Connecting to 195.154.53.219:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58196480 (56M) [application/octet-stream]
Saving to: ‘test.tar’

test.tar            100%[===================>]  55.50M  21.6MB/s    in 2.6s    

2022-10-30 19:46:35 (21.6 MB/s) - ‘test.tar’ saved [58196480/58196480]

tar: test_data: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [29]:
ds = load_dataset("imagefolder", data_dir = "./FIRE_DATABASE_3")
ds

Resolving data files:   0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 1500
    })
})

In [30]:
data = ds['train'].train_test_split(test_size = 0.1)
data

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 1350
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 150
    })
})

In [31]:
labels = data['train'].features['label']
labels = data["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

In [32]:
metric = load_metric('accuracy')

In [33]:
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

loading feature extractor configuration file https://huggingface.co/google/vit-base-patch16-224-in21k/resolve/main/preprocessor_config.json from cache at /root/.cache/huggingface/transformers/7c7f3e780b30eeeacd3962294e5154788caa6d9aa555ed6d5c2f0d2c485eba18.c322cbf30b69973d5aae6c0866f5cba198b5fe51a2fe259d2a506827ec6274bc
Feature extractor ViTFeatureExtractor {
  "do_normalize": true,
  "do_resize": true,
  "feature_extractor_type": "ViTFeatureExtractor",
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "size": 224
}



In [34]:
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
train_transforms = Compose(
        [
            RandomResizedCrop(feature_extractor.size),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )

val_transforms = Compose(
        [
            Resize(feature_extractor.size),
            CenterCrop(feature_extractor.size),
            ToTensor(),
            normalize,
        ]
    )

def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [
        train_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

In [35]:
train_ds = data['train']
val_ds = data['test']
test_ds = load_dataset("imagefolder", data_dir = "./FIRE_DATABASE_3")['train']

Resolving data files:   0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [36]:
test_ds

Dataset({
    features: ['image', 'label'],
    num_rows: 1500
})

In [47]:
train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)
test_ds.set_transform(preprocess_val)

In [38]:
train_ds[0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1280x960>,
 'label': 0,
 'pixel_values': tensor([[[-0.0980, -0.0980, -0.0745,  ..., -0.0667, -0.0745, -0.0824],
          [-0.0902, -0.0824, -0.0588,  ..., -0.0588, -0.0667, -0.0745],
          [-0.0588, -0.0745, -0.0588,  ..., -0.0431, -0.0510, -0.0667],
          ...,
          [-0.3020, -0.3569, -0.3961,  ..., -0.5137, -0.4039, -0.2784],
          [-0.2863, -0.3255, -0.3647,  ..., -0.5294, -0.4353, -0.3176],
          [-0.2863, -0.3255, -0.3569,  ..., -0.5373, -0.4667, -0.3647]],
 
         [[-0.7569, -0.7569, -0.7333,  ..., -0.7176, -0.7569, -0.7647],
          [-0.7569, -0.7490, -0.7255,  ..., -0.7176, -0.7412, -0.7490],
          [-0.7412, -0.7490, -0.7333,  ..., -0.7176, -0.7255, -0.7412],
          ...,
          [-0.8667, -0.8824, -0.9059,  ..., -0.9765, -0.9373, -0.8824],
          [-0.8510, -0.8667, -0.8902,  ..., -0.9765, -0.9451, -0.8824],
          [-0.8588, -0.8667, -0.8824,  ..., -0.9843, -0.9529, -0.8824]

In [39]:
model_name_or_path = 'google/vit-base-patch16-224-in21k'
model = ViTForImageClassification.from_pretrained(
    model_name_or_path, 
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

loading configuration file https://huggingface.co/google/vit-base-patch16-224-in21k/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/7bba26dd36a6ff9f6a9b19436dec361727bea03ec70fbfa82b70628109163eaa.92995a56e2eabab0c686015c4ad8275b4f9cbd858ed228f6a08936f2c31667e7
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224-in21k",
  "architectures": [
    "ViTModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "fire",
    "1": "no_fire",
    "2": "start_fire"
  },
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "fire": "0",
    "no_fire": "1",
    "start_fire": "2"
  },
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.20.1"
}

load

In [40]:
training_args = TrainingArguments(
    'finetuned-fire-detection',
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
  hub_strategy="end"
)#

PyTorch: setting up devices


In [41]:
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [42]:
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

In [43]:
from transformers import Trainer
trainer = Trainer(
    model,
    training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

Using cuda_amp half precision backend


In [44]:
train_results = trainer.train()
# rest is optional but nice to have
torch.save(model.state_dict(), "model.pth")
trainer.save_model('model.h5')
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1350
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 340


Step,Training Loss,Validation Loss,Accuracy
100,0.117600,0.070909,0.993333
200,0.132200,0.039215,0.993333
300,0.118100,0.068777,0.980000


***** Running Evaluation *****
  Num examples = 150
  Batch size = 8
Saving model checkpoint to finetuned-fire-detection/checkpoint-100
Configuration saved in finetuned-fire-detection/checkpoint-100/config.json
Model weights saved in finetuned-fire-detection/checkpoint-100/pytorch_model.bin
Feature extractor saved in finetuned-fire-detection/checkpoint-100/preprocessor_config.json
Deleting older checkpoint [finetuned-fire-detection/checkpoint-200] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 150
  Batch size = 8
Saving model checkpoint to finetuned-fire-detection/checkpoint-200
Configuration saved in finetuned-fire-detection/checkpoint-200/config.json
Model weights saved in finetuned-fire-detection/checkpoint-200/pytorch_model.bin
Feature extractor saved in finetuned-fire-detection/checkpoint-200/preprocessor_config.json
Deleting older checkpoint [finetuned-fire-detection/checkpoint-300] due to args.save_total_limit
***** Running Evaluation *****
  Num e

***** train metrics *****
  epoch                    =         4.0
  total_flos               = 389721705GF
  train_loss               =      0.1623
  train_runtime            =  0:03:49.67
  train_samples_per_second =      23.512
  train_steps_per_second   =        1.48


In [45]:
metrics = trainer.evaluate()
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** Running Evaluation *****
  Num examples = 150
  Batch size = 8


***** eval metrics *****
  epoch                   =        4.0
  eval_accuracy           =     0.9933
  eval_loss               =     0.0392
  eval_runtime            = 0:00:04.17
  eval_samples_per_second =     35.894
  eval_steps_per_second   =      4.547


In [48]:
outputs = trainer.predict(test_ds)
print(outputs.metrics)

***** Running Prediction *****
  Num examples = 1500
  Batch size = 8


{'test_loss': 0.04682157561182976, 'test_accuracy': 0.9906666666666667, 'test_runtime': 43.0964, 'test_samples_per_second': 34.806, 'test_steps_per_second': 4.362}
